In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

In [3]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp:int) -> str:
        return 'ComputeClass: ' + tmp + 'd'

In [4]:
def input_handler(i):
    return [i] * 2

In [5]:
out_queue= ppq.ThreadQueue().register_callback(print)
temp = partials(print, 'test')

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), input_handler, distributor),
            (input_handler, compute, ppq.ThreadQueue()),
                (compute, print, ppq.ThreadQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.ThreadQueue()),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.ThreadQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

[ThreadQueue(name=None, runner=1, id=127231265827216)]

  |->input_handler(i)->Any(id=127231266071840)-IteratorDistributor

    |->[ThreadQueue(name=None, runner=1, id=127231260011536)]

      |->compute(x)->Any(id=127231266072960)

        |->[ThreadQueue(name=None, runner=1, id=127231266113696)]

          |->print(*args, sep=str, end=str, file=NoneType, flush=bool)->Any(id=127231740994608)

        |->[ThreadQueue(name=None, runner=1, id=127231275389280)]

        |->[ThreadQueue(name=None, runner=1, id=127231266113088)]

          |->compute2(x)->Any(id=127231264295648)

            |->[ThreadQueue(name=None, runner=1, id=127231266113696)]

              |->print(*args, sep=str, end=str, file=NoneType, flush=bool)->Any(id=127231740994608)...

            |->[ThreadQueue(name=None, runner=1, id=127231275389280)]

            |->[ThreadQueue(name=None, runner=1, id=127231259834800)]

              |->ComputeClass[(tmp:int)->str](id=127231259961936)

                |->[ThreadQueue(na

In [ ]:
with flow:
   for i in range(10):
      flow.put(i)

In [ ]:
flow.alive()

In [ ]:
out_queue.run()

In [ ]:
out_queue.release()